In [1]:
import pandas as pd
import numpy as np
import json

from nltk.tokenize import wordpunct_tokenize
from models import *
from utils import *
from fuzzy_match import match

from pymorphy2 import MorphAnalyzer

In [2]:
pd.options.display.max_rows = 1000

In [3]:
vocab, w2v = load_vectors('vectors/MS_w2v_vectors.txt')
_, fasttext = load_vectors('vectors/MS_fasttext_vectors.txt')

In [3]:
len(vocab)

256515

In [4]:
class MultScler():

    def __init__(self, text, vocab, w2v, fasttext):
        
        self.text = text
        self.data = pd.DataFrame(wordpunct_tokenize(text.lower()), columns = ['token'])
        
        self.dmd_model = DMDtagger(embedding_dim = 100, hidden_dim = 128, char_vocab_size = 174, tagset_size = 3)
        self.dmd_model.load_state_dict(torch.load('model_dicts/DMD_best_model_state_dict.pth',
                                                   map_location=torch.device('cpu')))
        self.dmd_model.eval()
        
        self.intake_model = Intaketagger(embeddings = w2v, embedding_dim = 300, hidden_dim = 256, tagset_size = 3)
        self.intake_model.load_state_dict(torch.load('model_dicts/INTAKE_best_model_state_dict.pth',
                                                   map_location=torch.device('cpu')))
        self.intake_model.eval()
        
        self.adr_model =  ADRtagger(embeddings = torch.cat((w2v, fasttext), 1), embedding_dim = 600,
                                    hidden_dim = 256, tagset_size = 27)
        self.adr_model.load_state_dict(torch.load('model_dicts/ADR_best_model_state_dict.pth',
                                                   map_location=torch.device('cpu')))
        self.adr_model.eval()
        
        self.link_model = Linktagger(embeddings = torch.cat((w2v, fasttext), 1), embedding_dim = 600,
                                     hidden_dim = 256, tagset_size = 2)
        
        self.link_model.load_state_dict(torch.load('model_dicts/LINK_best_model_state_dict.pth',
                                                   map_location=torch.device('cpu')))
        self.link_model.eval()
        
        self.token_to_ind = vocab
        self.w2v = w2v
        self.fasttext = fasttext
        
        self.char_to_ind = load_json('json/char_to_ind.json')
        self.dmd_to_ind = load_json('json/dmd_to_ind.json')
        self.ind_to_dmd = {v:k for k, v in self.dmd_to_ind.items()}
        self.int_label_to_ind = load_json('json/int_label_to_ind.json')
        self.ind_to_int_label = {v:k for k, v in self.int_label_to_ind.items()}
        self.lex_to_ind = load_json('json/lex_to_ind.json')
        self.adr_to_ind = load_json('json/adr_to_ind.json')
        self.ind_to_adr = {v:k for k, v in self.adr_to_ind.items()}
        
        self.dmd_recovery_dict = load_json('json/dmd_recovery_dict.json')
        
        
        self.verbs = read_lexicon('lex/Verbs.txt')
        self.adrs = read_lexicon('lex/Adrs.txt')
        self.symptoms = read_lexicon('lex/Symptoms.txt')
        self.anatomy = read_lexicon('lex/Anatomy.txt')
        
        self.morph = MorphAnalyzer()
    
    def assign_tag(self, norms):
        if norms & self.anatomy:
            return 'ANATOMY'
        elif norms & self.symptoms:
            return 'SYMPTOM'
        elif norms & self.verbs:
            return 'VERB'
        elif norms & self.adrs:
            return 'ADR'
        else:
            return 'O'
    
    def add_dmd_tags(self):
        tokens = self.data['token']
        chars = [[char for char in word] for word in tokens]
        chars = [[self.char_to_ind[char] for char in word] for word in tokens]
        padded_text = []
        for word in chars:
            word = word[:12]        
            word += [0] * (12 - len(word))
            padded_text.append(word)
        padded_text = [padded_text]
        chars = torch.tensor(padded_text).long()
        with torch.no_grad():
            pred = self.dmd_model(chars)
        pred = pred.squeeze(0)
        pred = np.argmax(pred.numpy(), axis=1)
        pred = [self.ind_to_dmd[tag] for tag in pred]
        self.data['dmd_tag'] = pred
        
    def add_lex_tags(self):
        self.data['norm'] = self.data['token'].apply(lambda x: all_norms(x, self.morph))
        self.data['lex_tag'] = self.data['norm'].apply(self.assign_tag)
        self.data = self.data.drop(columns=['norm'])
    
    def prepare_data(self):
        self.add_dmd_tags()
        self.add_lex_tags()
        
    def define_intakes(self):
        tokens = self.data['token']
        tokens = [self.token_to_ind[token] for token in tokens]
        tokens = torch.tensor([tokens]).long()
        dmd_tags = self.data['dmd_tag']
        dmd_tags = [self.dmd_to_ind[dmd] for dmd in dmd_tags]
        dmd_tags = torch.tensor([dmd_tags]).long()
        with torch.no_grad():
            pred = self.intake_model(tokens, dmd_tags)
        pred = pred.squeeze(0)
        pred = np.argmax(pred.numpy(), axis=1)
        pred = [self.ind_to_int_label[pr] for pr in pred]
        self.data['label'] = pred
    
    def create_dmd_json(self):
        self.intake_cases = create_dmd_json(self.data, self.dmd_recovery_dict)
    
    def define_adrs(self):
        tokens = self.data['token']
        tokens = [self.token_to_ind[token] for token in tokens]
        tokens = torch.tensor([tokens]).long()
        lex_tags = self.data['lex_tag']
        lex_tags = [self.lex_to_ind[tag] for tag in lex_tags]
        lex_tags = torch.tensor([lex_tags]).long()
        with torch.no_grad():
            pred = self.adr_model(tokens, lex_tags)
        pred = pred.squeeze(0)
        pred = np.argmax(pred.numpy(), axis=1)
        pred = [self.ind_to_adr[pr] for pr in pred]
        self.data['adr_label'] = pred
        
    def add_adr_tags(self):
        self.data['adr_tag'] = 0
        self.data.loc[self.data['adr_label'] != 'O', 'adr_tag'] = 1
    
    def add_drug_tags(self):
        self.data['drug_tag'] = 0
        self.data.loc[self.data['dmd_tag'] != 'O', 'drug_tag'] = 2    
        
        
    def link_adrs(self):
        for i in range(len(self.intake_cases)):
            cur_dmd_inds = self.intake_cases[i]['inds']
            self.data.iloc[cur_dmd_inds, self.data.columns.get_loc('drug_tag')] = 1
            
            tokens = self.data['token']
            tokens = [self.token_to_ind[token] for token in tokens]
            tokens = torch.tensor([tokens]).long()
            
            adr_tags = self.data['adr_tag']
            adr_tags = torch.tensor([adr_tags]).long()
            
            drug_tags = self.data['drug_tag']
            drug_tags = torch.tensor([drug_tags]).long()
            with torch.no_grad():
                pred = self.link_model(tokens, adr_tags, drug_tags)
            pred = pred.squeeze(0)
            pred = np.argmax(pred.numpy(), axis=1)
            self.intake_cases[i]['adr_inds'] = np.nonzero(pred)[0].tolist()
            
            self.data.loc[self.data['drug_tag'] != 0, 'drug_tag'] = 2
    
    def add_adrs_to_dmd_json(self):
        for i in range(len(self.intake_cases)):
            adrs_dict = prepare_all_adr_tokens(self.intake_cases[i]['adr_inds'], ms.data)
            self.intake_cases[i]['adr_items'] = adrs_dict
    
    def parse_text(self):
        self.prepare_data()
        self.define_intakes()
        self.create_dmd_json()
        if len(self.intake_cases) > 0:
            self.define_adrs()
            self.add_adr_tags()
            self.add_drug_tags()
            self.link_adrs()
            self.add_adrs_to_dmd_json()
        self.print_report()
    
    def replace_text(self, new_text):
        self.text = new_text
        self.data = pd.DataFrame(wordpunct_tokenize(new_text.lower()), columns = ['token'])
        
    def print_report(self):
        print(f'Всего событий приема ПИТРС обнаружено: {len(self.intake_cases)}')
        for i in range(len(self.intake_cases)):
            print('----')
            print(f'{i+1}:')
            print(f"Наименование препарата: {self.intake_cases[i]['full_dmd_name'].capitalize()}\n")
            print('Выявленные побочные эффекты:')
            for j in range(len(self.intake_cases[i]['adr_items'])):
                print(f"{self.intake_cases[i]['adr_items'][j]['class']}: {' '.join(self.intake_cases[i]['adr_items'][j]['tokens'])}")
            

In [157]:
text = 'Слоники, всем привет и долгой ремиссии! Подскажите, я принимаю Абаджио, сильно выпадают волосы...'

In [158]:
ms = MultScler(text, vocab, w2v, fasttext)

In [165]:
ms.replace_text('Я тоже с такими же вопросами начала колоть коп! проблемы у меня только с глазами. По симптоматике никаких изменений нет(тьфу тьфу тьфу), по мрт буду смотреть в конце лета, волнуюсь немного. Колю коп почти год, сейчас начала аксоглатиран. Из побочек только зуд, но мажу фенистилом, один раз после укола стало сердце как бешеное биться. На время остановила уколы)) сейчас колю дальше чередуя коп и аксо)) думаю у вас будет все нормальножелаю удачи!')

In [166]:
ms.parse_text()

Всего событий приема ПИТРС обнаружено: 1
----
1:
Наименование препарата: Аксоглатиран

Выявленные побочные эффекты:
FLS: зуд
LR: стало сердце как бешеное биться


In [ ]:
ms.data

In [167]:
ms.data

,token,dmd_tag,lex_tag,label,adr_label,adr_tag,drug_tag
0,я,O,O,O,O,0,0
1,тоже,O,O,O,O,0,0
2,с,O,O,O,O,0,0
3,такими,O,O,O,O,0,0
4,же,O,O,O,O,0,0
5,вопросами,O,O,O,O,0,0
6,начала,O,O,B-INT,O,0,0
7,колоть,O,O,I-INT,O,0,0
8,коп,O,O,I-INT,O,0,0
9,!,O,O,O,O,0,0


In [160]:
ms.intake_cases

{0: {'dmd_names': ['абаджио'],
  'inds': [12],
  'full_dmd_name': 'абаджио',
  'adr_inds': [15, 16],
  'adr_items': [{'class': 'IS',
    'tokens': ['выпадают', 'волосы'],
    'inds': [15, 16]}]}}

In [146]:
examples = list(read_lexicon('examples.txt')) 

In [151]:
for i, text in enumerate(examples):
    print(f'--- Пример {i+1} ---')
    print(text)
    print()
    ms.replace_text(text)
    ms.parse_text()
    print()

--- Пример 1 ---
А мне Авонекс больше нравился, с Синновексом мне стало хуже, и его заменили Тимексоном, уже 2 года колю каждый день, пока тоже нехорошо, но врач отказывается переводить обратно на интерфероны 😡.

Всего событий приема ПИТРС обнаружено: 4
----
1:
Наименование препарата: Авонекс

Выявленные побочные эффекты:
----
2:
Наименование препарата: Синновекс

Выявленные побочные эффекты:
----
3:
Наименование препарата: Тимексон

Выявленные побочные эффекты:
UNK: нехорошо
----
4:
Наименование препарата: Интерферон

Выявленные побочные эффекты:

--- Пример 2 ---
я принимала.от феморикса волосы сильно выпадывали, а вот от  абаджио и терифлуномида побочек не было.

Всего событий приема ПИТРС обнаружено: 3
----
1:
Наименование препарата: Феморикс

Выявленные побочные эффекты:
IS: волосы сильно выпадывали
----
2:
Наименование препарата: Абаджио

Выявленные побочные эффекты:
NEG: побочек не было
----
3:
Наименование препарата: Терифлуномид

Выявленные побочные эффекты:
NEG: побочек не бы

Всего событий приема ПИТРС обнаружено: 1
----
1:
Наименование препарата: Интерферон бета-1b

Выявленные побочные эффекты:
UNK: побочные
FLS: гриппозный трусит жутко
FLS: температура повышается выше 38 , до 39
FLS: температуры
FLS: озноба
FLS: температура выше 38
FLS: температура 38 . 6
FLS: жутко руки
FLS: нос
FLS: ноги мёрзнут
FLS: температуры
NS: сильная усталость
FLS: ознобом
FLS: температурой
FLS: болит спина
FLS: температура , около 37 . 2
MSS: боль в спине
FLS: температура высокая
FLS: озноб
FLS: жар
FLS: температуру болит

--- Пример 17 ---
Всем привет) Месяц назад уже писала сюда большой пост, когда пришла сюда новенькой, вот появились еще вопросы, поможете?)1) Кажется, от Тимексона начали сильно лезть волосы. У кого было? Это пройдет? Чем лечили волосики?Показать полностью... 2) Что кому помогает от синяков после инъекций? Использую Троксерутин, но что-то как-то...никак 3) Кто в каких дозах принимает витамин Д? У меня результат 33 (норма 30-100) Слышала от нескольких врачей ве

In [147]:
examples

['А мне Авонекс больше нравился, с Синновексом мне стало хуже, и его заменили Тимексоном, уже 2 года колю каждый день, пока тоже нехорошо, но врач отказывается переводить обратно на интерфероны 😡.',
 'я принимала.от феморикса волосы сильно выпадывали, а вот от  абаджио и терифлуномида побочек не было.',
 'Вчера самостоятельно отменила себе Инфабету 1 б. Сил нет терпеть побочки: мигрень, которая появилась как стала ее колоть и слабость ещё больше, депрессия, потеря веса. Раньше обострений особо не было, колола Копаксон, а тут,ну просто капец. Стала бешеная, плаксивая и слабая. Кто-то отменял самостоятельно? Спасибо за советы',
 'Я год колола Копаксон/ Тиран и весь год чесалась. А потом началась гигантская крапивница с отеками Квинке и меня сняли с инъекций.',
 'Блин у меня от Тизабри волосы прям вытаскиваю пучками... От Копа честно говоря не помню',
 'Я тоже с такими же вопросами начала колоть коп! проблемы у меня только с глазами. По симптоматике никаких изменений нет(тьфу тьфу тьфу), 

In [144]:
ms.parse_text()

Всего событий приема ПИТРС обнаружено: 0


In [143]:
ms.replace_text(text)